In [ ]:
#OBS: Código executado no colab
#Se você já tem o arquivo doc_clean_unstructed não precisa executar este arquivo
# Instala todas as bibliotecas necessárias para o pipeline
!pip install -qU langchain langchain_community langchain_core
!pip install -qU pandas beautifulsoup4
!pip install -qU "unstructured[local-inference]" unstructured_client
!pip install -qU langchain-google-genai qdrant-client pypdf

# A Unstructured precisa da biblioteca 'poppler' para processar PDFs.
# No Colab (que é um ambiente Linux), podemos instalar com o comando abaixo.
!apt-get install poppler-utils

In [ ]:
import json
import re
from typing import List

from langchain_core.documents import Document
from langchain_community.document_loaders import CSVLoader, UnstructuredURLLoader

# ======================================================
# === FUNÇÕES DE CACHE (Salvar e Carregar JSONL) =======
# ======================================================

def salvar_documentos_em_jsonl(documentos: List, caminho_arquivo: str):
    """Salva uma lista de Documentos LangChain em um arquivo.jsonl."""
    with open(caminho_arquivo, 'w', encoding='utf-8') as f:
        for doc in documentos:
            linha = json.dumps({
                'page_content': doc.page_content,
                'metadata': doc.metadata
            }, ensure_ascii=False)
            f.write(linha + '\n')
    print(f"Documentos salvos em: {caminho_arquivo} ({len(documentos)} registros)")

def carregar_documentos_de_jsonl(caminho_arquivo: str) -> List:
    """Carrega uma lista de Documentos LangChain de um arquivo.jsonl."""
    documentos = []
    with open(caminho_arquivo, 'r', encoding='utf-8') as f:
        for linha in f:
            dados = json.loads(linha)
            doc = Document(page_content=dados['page_content'], metadata=dados['metadata'])
            documentos.append(doc)
    print(f"{len(documentos)} documentos carregados do cache: {caminho_arquivo}")
    return documentos

# ======================================================
# === FUNÇÃO DE LIMPEZA FINAL DE TEXTO =================
# ======================================================

def limpar_texto_final(texto: str) -> str:
    """Remove múltiplos espaços e quebras de linha para um texto já limpo."""
    texto = re.sub(r'\s+', ' ', texto)
    texto = texto.strip()
    return texto

# ======================================================
# === FUNÇÃO PRINCIPAL DE COLETA E LIMPEZA
# ======================================================

def carregar_e_limpar_conteudo(documentos_origem: List) -> List:
    """
    Faz o download e a limpeza inteligente de cada documento usando UnstructuredURLLoader.
    """
    print("Iniciando o processo de download e limpeza inteligente com Unstructured...")
    documentos_finais = []
    urls_processadas = set()

    # Contadores de tipo de documento
    total_pdfs = 0
    total_htmls = 0
    total_processadas = 0

    for i, doc_origem in enumerate(documentos_origem):
        source_url = doc_origem.metadata['source']
        if source_url in urls_processadas:
            print(f"⚠️ URL repetida ignorada: {source_url}")
            continue
        urls_processadas.add(source_url)
        print(f"Processando URL {i+1}/{len(documentos_origem)}: {source_url}")

        try:
            # 1. Adicionamos o parâmetro 'languages' para otimizar para o português.
            # 2. Usamos a estratégia 'hi_res' para PDFs para maior precisão.
            is_pdf = source_url.lower().endswith(".pdf") or "format=pdf" in source_url.lower()
            strategy = "hi_res" if '.pdf' in source_url.lower() else "fast"

            loader = UnstructuredURLLoader(
                urls=[source_url],
                mode="elements",
                strategy=strategy,
                unstructured_kwargs={"languages": ["por"]} # <-- INFORMA O IDIOMA
            )
            elementos = loader.load()

            # Em vez de permitir apenas alguns tipos, vamos excluir os tipos irrelevantes.
            tipos_irrelevantes = ["Header", "Footer", "PageNumber", "Image", "Table", "FigureCaption"]


            texto_combinado = ""
            for elemento in elementos:
                # Mantemos o elemento se sua categoria NÃO ESTIVER na lista de irrelevantes
                if elemento.metadata.get('category') not in tipos_irrelevantes:
                    texto_combinado += elemento.page_content + "\n\n"

            if len(texto_combinado) > 100:
                texto_final_limpo = limpar_texto_final(texto_combinado)
                metadata_final = {
                    **doc_origem.metadata,
                    "tipo": "PDF" if is_pdf else "HTML",
                    "num_palavras": len(texto_final_limpo.split())
                }
                documentos_finais.append(Document(
                    page_content=texto_final_limpo,
                    metadata=doc_origem.metadata
                ))

                # Incrementa contadores
                if is_pdf:
                    total_pdfs += 1
                else:
                    total_htmls += 1

                total_processadas += 1

        except Exception as e:
            print(f"-> Erro ao processar {source_url} com Unstructured: {e}")
            continue

    print(f"✅ {len(documentos_finais)} documentos coletados e limpos com sucesso.")

    # ======================================================
    # === RELATÓRIO FINAL ==================================
    # ======================================================
    print("\n✅ Coleta e limpeza concluídas!")
    print(f"🔗 Total de URLs únicas analisadas: {len(urls_processadas)}")
    print(f"📄 PDFs processados com sucesso: {total_pdfs}")
    print(f"🌐 Páginas web processadas com sucesso: {total_htmls}")
    print(f"🧾 Total de URLs com sucesso (conteúdo válido): {total_processadas}")
    print(f"⚙️ Documentos válidos salvos: {len(documentos_finais)}")

    # Exibe porcentagens
    if total_processadas > 0:
        perc_pdfs = (total_pdfs / total_processadas) * 100
        perc_htmls = (total_htmls / total_processadas) * 100
        print(f"📊 Distribuição: {perc_pdfs:.1f}% PDFs | {perc_htmls:.1f}% Web")
    return documentos_finais

print("Funções do pipeline (versão corrigida e mais inteligente) definidas com sucesso!")

In [ ]:
# Certifique-se de que você já fez o upload do arquivo 'articles.csv'
# usando o ícone de pasta na barra lateral esquerda.

csv_file_path = 'articles.csv'
print(f"O script agora irá usar o arquivo '{csv_file_path}' que você enviou.")

In [ ]:
#OBS: Essa parte demora em média 1793.81 segundos
import os
import time

try:
    from google.colab import files
    em_colab = True
except ImportError:
    em_colab = False

inicio_execucao = time.time()
# Fase I: Ler lista de URLs do CSV
if 'csv_file_path' in locals() and csv_file_path:
    loader = CSVLoader(
        file_path=csv_file_path,
        source_column="link",
        encoding='utf-8'
    )
    docs_from_csv = loader.load()
    print(f"Fase I concluída: {len(docs_from_csv)} URLs carregados de '{csv_file_path}'.")

    # Fase II: Cache e limpeza inteligente
    arquivo_cache_fase2 = 'documentos_limpos_unstructured.jsonl'

    if os.path.exists(arquivo_cache_fase2):
        documentos_carregados_e_limpos = carregar_documentos_de_jsonl(arquivo_cache_fase2)
    else:
        documentos_carregados_e_limpos = carregar_e_limpar_conteudo(docs_from_csv)
        if documentos_carregados_e_limpos:
            salvar_documentos_em_jsonl(documentos_carregados_e_limpos, arquivo_cache_fase2)

    # Exemplo de documento limpo
    if documentos_carregados_e_limpos:
        doc_exemplo = documentos_carregados_e_limpos
        print("\n--- Exemplo de Documento Limpo (pós-Unstructured) ---")
        print(f"Conteúdo (primeiros 500 caracteres): {doc_exemplo[0]}...")
        print("----------------------------------------------------")

    # Disponibilizar arquivo para download se estiver no Colab
    if em_colab and os.path.exists(arquivo_cache_fase2):
        print("\n📥 Fazendo download do arquivo gerado...")
        files.download(arquivo_cache_fase2)
else:
    print("Por favor, execute a Célula 3 para fazer o upload do arquivo CSV primeiro.")

# Marcar o fim da execução
fim_execucao = time.time()
tempo_total = fim_execucao - inicio_execucao

print(f"\n⏱️ Tempo total de execução: {tempo_total:.2f} segundos.")